In [2]:
import pandas as pd
import pathlib
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import re
from datetime import datetime

In [5]:
current_path = os.getcwd()
data_path = '../data/San_Francisco_Population_and_Demographic_Census_Data_20250729.csv'
full_path = os.path.join(current_path, data_path)

In [6]:
df = pd.read_csv(full_path)


In [42]:
print(f"Original shape: {df.shape}")
print(f"Original unique geographies: {df['geography_id'].nunique()}")

Original shape: (31127, 32)
Original unique geographies: 340


In [43]:
df.columns

Index(['source', 'num_years', 'start_year', 'end_year', 'derived',
       'derived_details', 'geography', 'geography_id_name', 'geography_id',
       'geography_name', 'demographic_category', 'demographic_category_label',
       'min_age', 'max_age', 'unit', 'estimate', 'se', 'moe', 'cv', 'reliable',
       'upper_ci', 'lower_ci', 'acs_table', 'acs_code', 'acs_label',
       'acs_concept', 'pums_variables', 'overall_segment', 'reporting_segment',
       'row_identifier', 'data_as_of', 'data_loaded_at'],
      dtype='object')

In [44]:
df.head()

,source,num_years,start_year,end_year,derived,derived_details,geography,geography_id_name,geography_id,geography_name,...,acs_table,acs_code,acs_label,acs_concept,pums_variables,overall_segment,reporting_segment,row_identifier,data_as_of,data_loaded_at
0,PUMS_acs5,5,2014,2019,True,"from public use micro data, se calculated usin...",county,GEOID,6075.0,"San Francisco County, California",...,NaN,NaN,NaN,NaN,AGEP,PUMS_acs5 population count data from 2014 to 2...,COVID-19 vaccine reporting - age groups,PUMS_acs5 population count data from 2014 to 2...,10/20/2022 02:24:33 PM,10/20/2022 02:28:00 PM
1,PUMS_acs5,5,2015,2020,True,"from public use micro data, se calculated usin...",county,GEOID,6075.0,"San Francisco County, California",...,NaN,NaN,NaN,NaN,AGEP,PUMS_acs5 population count data from 2015 to 2...,COVID-19 vaccine reporting - age groups,PUMS_acs5 population count data from 2015 to 2...,10/20/2022 02:24:34 PM,10/20/2022 02:28:00 PM
2,PUMS_acs5,5,2014,2019,True,"from public use micro data, se calculated usin...",county,GEOID,6075.0,"San Francisco County, California",...,NaN,NaN,NaN,NaN,AGEP,PUMS_acs5 population count data from 2014 to 2...,COVID-19 vaccine reporting - age groups,PUMS_acs5 population count data from 2014 to 2...,10/20/2022 02:24:35 PM,10/20/2022 02:28:00 PM
3,PUMS_acs5,5,2015,2020,True,"from public use micro data, se calculated usin...",county,GEOID,6075.0,"San Francisco County, California",...,NaN,NaN,NaN,NaN,AGEP,PUMS_acs5 population count data from 2015 to 2...,COVID-19 vaccine reporting - age groups,PUMS_acs5 population count data from 2015 to 2...,10/20/2022 02:24:36 PM,10/20/2022 02:28:00 PM
4,acs5,5,2016,2021,False,from acs table,county,GEOID,6075.0,"San Francisco County, California",...,B01001,1.0,Estimate!!Total:,SEX BY AGE,NaN,acs5 population count data from 2016 to 2021 a...,from census table B01001,acs5 population count data from 2016 to 2021 a...,03/06/2023 12:09:00 PM,03/06/2023 12:32:00 PM


### ////


In [45]:
age_df = df.copy()
age_df = age_df[age_df['estimate'].notna()].copy()

# Convert geography_id to string (remove .0)
age_df['geography_id'] = age_df['geography_id'].astype(str).str.replace('.0', '', regex=False)

# Create time_key
age_df['time_key'] = age_df['start_year'].astype(str) + '_' + age_df['end_year'].astype(str)

print(f"Rows after filtering to AGEP (age) and valid estimate: {len(age_df)}")
print(f"Geographies after filter: {age_df['geography_id'].nunique()}")


Rows after filtering to AGEP (age) and valid estimate: 31119
Geographies after filter: 341


In [46]:
def clean_text(x):
    if pd.isna(x):
        return "Unknown"
    x = str(x).strip()
    x = unidecode(x)  # Remove accents
    x = re.sub(r'\s+', ' ', x)  # Normalize whitespace
    return x.lower()

# Apply to key fields
age_df['reporting_segment'] = age_df['reporting_segment'].apply(clean_text)
age_df['overall_segment'] = age_df['overall_segment'].apply(clean_text)
age_df['acs_concept'] = age_df['acs_concept'].apply(clean_text)

In [47]:
age_df.shape

(31119, 33)

In [48]:
# 1. Source_Dim
source_dim = df[['source', 'derived', 'derived_details']].drop_duplicates()
source_dim['source_key'] = range(1, len(source_dim) + 1)
source_dim = source_dim.rename(columns={
    'source': 'source_name',
    'derived': 'derived_flag'
})

print(f"Created Source_Dim with {len(source_dim)} records")

Created Source_Dim with 9 records


In [49]:
geography_dim = df[['geography', 'geography_id_name', 'geography_id', 'geography_name']].drop_duplicates()
geography_dim['geography_key'] = range(1, len(geography_dim) + 1)

print(f"Created Geography_Dim with {len(geography_dim)} records")

Created Geography_Dim with 920 records


In [50]:
geography_dim

,geography,geography_id_name,geography_id,geography_name,geography_key
0,county,GEOID,6.075000e+03,"San Francisco County, California",1
14,tract,GEOID,6.075010e+09,"Census Tract 101.01, San Francisco County, Cal...",2
15,tract,GEOID,6.075010e+09,"Census Tract 101.02, San Francisco County, Cal...",3
16,tract,GEOID,6.075010e+09,"Census Tract 102.01, San Francisco County, Cal...",4
17,tract,GEOID,6.075010e+09,"Census Tract 102.02, San Francisco County, Cal...",5
...,...,...,...,...,...
23018,tract,GEOID,6.075048e+09,Census Tract 479.01,916
23023,tract,GEOID,6.075061e+09,Census Tract 607,917
23025,tract,GEOID,6.075061e+09,Census Tract 611,918
23027,tract,GEOID,6.075061e+09,Census Tract 614,919


In [ ]:
# 3. Time_Dim
time_dim = df[['start_year', 'end_year']].drop_duplicates()
time_dim['time_key'] = range(1, len(time_dim) + 1)
time_dim['period_length'] = time_dim['end_year'] - time_dim['start_year']
print(f"Optimized Time_Dim shape: {time_dim.shape}")

# 4. Segment_Dim
segment_dim = df[['overall_segment', 'reporting_segment']].drop_duplicates()
segment_dim['segment_key'] = range(1, len(segment_dim) + 1)
# segment_dim.to_sql('Segment_Dim', conn, index=False, if_exists='replace')
print(f"Created Segment_Dim with {len(segment_dim)} records")

# 5. Variable_Dim
variable_dim = df[['pums_variables', 'acs_table', 'acs_code', 'acs_label', 'acs_concept']].drop_duplicates()
variable_dim['variable_key'] = range(1, len(variable_dim) + 1)
# variable_dim.to_sql('Variable_Dim', conn, index=False, if_exists='replace')
print(f"Created Variable_Dim with {len(variable_dim)} records")



Optimized Time_Dim shape: (17, 4)
Created Segment_Dim with 165 records
Created Variable_Dim with 65 records
Created DataQuality_Dim with 1 records


In [ ]:
fact_table = df.copy()

# Add foreign keys by merging with dimension tables
fact_table = fact_table.merge(
    source_dim[['source_key', 'source_name', 'derived_flag', 'derived_details']], 
    left_on=['source', 'derived', 'derived_details'], 
    right_on=['source_name', 'derived_flag', 'derived_details'], 
    how='left'
)

fact_table = fact_table.merge(
    geography_dim[['geography_key', 'geography', 'geography_id_name', 'geography_id', 'geography_name']], 
    on=['geography', 'geography_id_name', 'geography_id', 'geography_name'], 
    how='left'
)

# In fact table creation, modify the time merge:
fact_table = fact_table.merge(
    time_dim[['time_key', 'start_year', 'end_year']], 
    on=['start_year', 'end_year'],  # Remove precise timestamps from join
    how='left'
)

fact_table = fact_table.merge(
    segment_dim[['segment_key', 'overall_segment', 'reporting_segment']], 
    on=['overall_segment', 'reporting_segment'], 
    how='left'
)

fact_table = fact_table.merge(
    variable_dim[['variable_key', 'pums_variables', 'acs_table', 'acs_code', 'acs_label', 'acs_concept']], 
    on=['pums_variables', 'acs_table', 'acs_code', 'acs_label', 'acs_concept'], 
    how='left'
)

# Add data quality key
fact_table['data_quality_key'] = 1

# Add fact_id
fact_table['fact_id'] = range(1, len(fact_table) + 1)

# Select final fact table columns
base_fact_columns = [
    'fact_id', 'source_key', 'geography_key', 'time_key', 
    'segment_key', 'variable_key', 'data_quality_key'
]

# Add numeric measures
numeric_columns = ['num_years', 'start_year', 'end_year', 'estimate', 'se', 'moe', 'cv', 'upper_ci', 'lower_ci']
available_numeric = [col for col in numeric_columns if col in fact_table.columns]

# Add boolean/reliable flag
flag_columns = ['reliable'] if 'reliable' in fact_table.columns else []

# Add metadata columns
metadata_columns = ['row_identifier', 'data_as_of', 'data_loafact_table_final_finalded_at']
available_metadata = [col for col in metadata_columns if col in fact_table.columns]

# Combine all columns
fact_columns = base_fact_columns + available_numeric + flag_columns + available_metadata

# Verify all columns exist
existing_columns = [col for col in fact_columns if col in fact_table.columns]
print(f"Available fact columns: {len(existing_columns)}/{len(fact_columns)}")
print(f"Missing columns: {[col for col in fact_columns if col not in fact_table.columns]}")

# Create final fact table
fact_table_final = fact_table[existing_columns]
# fact_table_final.to_sql('Population_Facts', conn, index=False, if_exists='replace')
print(f"Created Population_Facts with {len(fact_table_final)} records")

# # Create indexes for better performance
# cursor.execute("CREATE INDEX IF NOT EXISTS idx_fact_source ON Population_Facts(source_key)")
# cursor.execute("CREATE INDEX IF NOT EXISTS idx_fact_geography ON Population_Facts(geography_key)")
# cursor.execute("CREATE INDEX IF NOT EXISTS idx_fact_time ON Population_Facts(time_key)")

# # Commit changes and close connection
# conn.commit()
# conn.close()

# print(f"\nStar schema created successfully in {db_path}")

Available fact columns: 20/20
Missing columns: []
Created Population_Facts with 31127 records


In [53]:
print("source_dim.shape",source_dim.shape)
print("geography_dim.shape",geography_dim.shape)
print("time_dim.shape",time_dim.shape)
print("segment_dim.shape",segment_dim.shape)
print("variable_dim.shape",variable_dim.shape)

source_dim.shape (9, 4)
geography_dim.shape (920, 5)
time_dim.shape (17, 4)
segment_dim.shape (165, 3)
variable_dim.shape (65, 6)


In [54]:
fact_table_final.shape

(31127, 20)

In [55]:
fact_table_final.head()

,fact_id,source_key,geography_key,time_key,segment_key,variable_key,data_quality_key,num_years,start_year,end_year,estimate,se,moe,cv,upper_ci,lower_ci,reliable,row_identifier,data_as_of,data_loaded_at
0,1,1,1,1,1,1,1,5,2014,2019,757193.0,1148.231183,1888.840296,0.001516,759478.497000,754907.503000,True,PUMS_acs5 population count data from 2014 to 2...,10/20/2022 02:24:33 PM,10/20/2022 02:28:00 PM
1,2,1,1,2,2,1,1,5,2015,2020,757150.0,1552.237063,2553.429969,0.002050,760239.650589,754060.349411,True,PUMS_acs5 population count data from 2015 to 2...,10/20/2022 02:24:34 PM,10/20/2022 02:28:00 PM
2,3,1,1,1,1,1,1,5,2014,2019,292025.0,691.494866,1137.509055,0.002368,293401.386102,290648.613898,True,PUMS_acs5 population count data from 2014 to 2...,10/20/2022 02:24:35 PM,10/20/2022 02:28:00 PM
3,4,1,1,2,2,1,1,5,2015,2020,294363.0,1015.866945,1671.101125,0.003451,296385.032574,292340.967426,True,PUMS_acs5 population count data from 2015 to 2...,10/20/2022 02:24:36 PM,10/20/2022 02:28:00 PM
4,5,2,1,3,3,2,1,5,2016,2021,865933.0,0.000000,0.000000,0.000000,865933.000000,865933.000000,True,acs5 population count data from 2016 to 2021 a...,03/06/2023 12:09:00 PM,03/06/2023 12:32:00 PM


In [56]:
fact_table_final.isna().sum()

fact_id                0
source_key             0
geography_key          0
time_key               0
segment_key            0
variable_key           0
data_quality_key       0
num_years              0
start_year             0
end_year               0
estimate               8
se                    12
moe                   12
cv                  3438
upper_ci              20
lower_ci              20
reliable               0
row_identifier         0
data_as_of             0
data_loaded_at         0
dtype: int64

In [57]:
fact_table_final['cv'] = fact_table_final['cv'].fillna(-1) 

/tmp/ipykernel_219972/1675378536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table_final['cv'] = fact_table_final['cv'].fillna(-1)


In [58]:
fact_table_final.isna().sum()

fact_id              0
source_key           0
geography_key        0
time_key             0
segment_key          0
variable_key         0
data_quality_key     0
num_years            0
start_year           0
end_year             0
estimate             8
se                  12
moe                 12
cv                   0
upper_ci            20
lower_ci            20
reliable             0
row_identifier       0
data_as_of           0
data_loaded_at       0
dtype: int64

In [59]:
# Print columns in fact_table_final that have NaN values
nan_columns = fact_table_final.columns[fact_table_final.isna().any()]
print("Columns with NaNs:", list(nan_columns))

Columns with NaNs: ['estimate', 'se', 'moe', 'upper_ci', 'lower_ci']


In [62]:
for i in nan_columns:
    fact_table_final[i].fillna(-1, inplace=True)  

# Fill NaNs with -1 or any other default value

/tmp/ipykernel_219972/2633303697.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fact_table_final[i].fillna(-1, inplace=True)
/tmp/ipykernel_219972/2633303697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table_final[i].fillna(-1, inplace=True)


In [63]:
fact_table_final.isna().sum()

fact_id             0
source_key          0
geography_key       0
time_key            0
segment_key         0
variable_key        0
data_quality_key    0
num_years           0
start_year          0
end_year            0
estimate            0
se                  0
moe                 0
cv                  0
upper_ci            0
lower_ci            0
reliable            0
row_identifier      0
data_as_of          0
data_loaded_at      0
dtype: int64

In [69]:
dim_dfs = {
    "source_dim": source_dim,
    "geography_dim": geography_dim,
    "time_dim": time_dim,
    "segment_dim": segment_dim,
    "variable_dim": variable_dim
}

In [ ]:
fact_table_final.to_csv('../data/star_schema/fact_table_final.csv', index=False)

In [ ]:
for key, df in dim_dfs.items():
    df.to_csv(f'../data/star_schema/{key}.csv', index=False)
    print(f"Saved {key} with shape {df.shape} to CSV")

Saved source_dim with shape (9, 4) to CSV
Saved geography_dim with shape (920, 5) to CSV
Saved time_dim with shape (17, 4) to CSV
Saved segment_dim with shape (165, 3) to CSV
Saved variable_dim with shape (65, 6) to CSV
